<a href="https://colab.research.google.com/github/AbuzerXudaverdi/water-potability-project/blob/main/potability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
#from google.colab import files
#uploaded = files.upload()
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import io
url = 'https://raw.githubusercontent.com/AbuzerXudaverdi/water-potability-project/main/water_potability.csv'
df = pd.read_csv(url)

In [70]:
#import pandas as pd
#import io
#df = pd.read_csv(io.BytesIO(uploaded['water_potability.csv']))
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import plot_confusion_matrix
import numpy as np
print(df.head)

<bound method NDFrame.head of             ph    Hardness        Solids  Chloramines     Sulfate  \
0          NaN  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246         NaN   
2     8.099124  224.236259  19909.541732     9.275884         NaN   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3271  4.668102  193.681735  47580.991603     7.166639  359.948574   
3272  7.808856  193.553212  17329.802160     8.061362         NaN   
3273  9.419510  175.762646  33155.578218     7.350233         NaN   
3274  5.126763  230.603758  11983.869376     6.303357         NaN   
3275  7.874671  195.102299  17404.177061     7.509306         NaN   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       564.308654       10.379783        86.990970   2.963135  

In [71]:
#print(mat.keys())
#print(len(mat['features'][1]))
#print(mat['features'])
cols = []
for col in df.columns:
  if col == 'Potability':
    cols.append(col)

print("Null values of 'Sulfate' column:",sum(pd.isnull(df['Sulfate'])))
print("Null values of 'Turbidity' column:", sum(pd.isnull(df['Turbidity'])))
print("Null values of 'Trihalomethanes' column:", sum(pd.isnull(df['Trihalomethanes'])))
print("Null values of 'pH' column:", sum(pd.isnull(df['ph'])))
print("Null values of 'Conductivity' column:", sum(pd.isnull(df['Conductivity'])))
print("Null values of 'Organic_carbon' column:", sum(pd.isnull(df['Organic_carbon'])))
print("Null values of 'Potability' column:", sum(pd.isnull(df['Potability'])))
print("Null values of 'Hardness' column:", sum(pd.isnull(df['Hardness'])))
print("Null values of 'Solids' column:", sum(pd.isnull(df['Solids'])))
print("Null values of 'Chloramines' column:", sum(pd.isnull(df['Chloramines'])))
x = sum(pd.isnull(df['Sulfate']))/len(df['Sulfate'])
y = len(df['Sulfate'])
print (x, y)

Null values of 'Sulfate' column: 781
Null values of 'Turbidity' column: 0
Null values of 'Trihalomethanes' column: 162
Null values of 'pH' column: 491
Null values of 'Conductivity' column: 0
Null values of 'Organic_carbon' column: 0
Null values of 'Potability' column: 0
Null values of 'Hardness' column: 0
Null values of 'Solids' column: 0
Null values of 'Chloramines' column: 0
0.23840048840048841 3276


In [72]:
df2 = df.dropna(subset = ['ph','Sulfate', 'Trihalomethanes'])
# best feature set is 'Sulfate','Turbidity','Trihalomethanes','ph', 'Conductivity','Chloramines'
X = df2[['Sulfate','Turbidity','Trihalomethanes','ph', 'Conductivity','Chloramines']].copy()
#X = df2[[cols]]
y = df2[['Potability']].copy()

X = df[['Chloramines','Solids','Turbidity']].copy()

y = df['Potability'].copy()
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
model = LinearRegression()
model.fit(x_train,y_train)
preds = model.predict(x_test)
mse = mean_squared_error(y_test,preds)
print(mse) ---> 0.23420358208983516, 0.23409812743606703


In [73]:

#best test size is probably 0.14 for this case
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.14, random_state = 0)
model = KNeighborsClassifier(n_neighbors=50)
model.fit(x_train,y_train.values.ravel())
preds = model.predict(x_test)  # nump.ndarry
print(preds.shape, y_test.shape)
preds = preds.reshape(282,1)
print(y_test.head())
print(preds.shape, y_test.shape)
#print(preds.head())
x = np.array(preds)
yy = np.array(y_test)
#print(yy)
#print(x.astype)
np_y = y_test.to_numpy()
#print(np_y)
acc = np.sum(x==yy)/len(yy)
print(acc)
#acc = np.sum(preds==np_y)/len(np_y.values)
##print(acc)

(282,) (282, 1)
      Potability
367            1
1985           1
2962           0
710            1
1344           0
(282, 1) (282, 1)
0.6524822695035462


In [99]:
from sklearn.model_selection import KFold
x_training, x_testing, y_training, y_testing = train_test_split(X,y, test_size = 0.14, random_state = 0)
kf = KFold(n_splits=10,shuffle=False)
# List to store the n_neighbors value as you iterate through the range n_range
neighbor_vals = []
temp = []
n_range = [int(5),int(50)]
# List to store the mean accuracy for each n_neighbors value
results = []
# Iterate through the n_neighbors values and perform cross-validation
# Calculate the mean validation fold accuracy for each across all validation folds
for n in n_range:
  neighbor_vals.append(n)
  knn_model = KNeighborsClassifier(n_neighbors=n)
  for train, val in kf.split(X=x_training,y=y_training):
    X_fold_train, X_fold_val = x_training.iloc[train], x_training.iloc[val]
    y_fold_train, y_fold_val = y_training.iloc[train], y_training.iloc[val]

    knn_model.fit( X_fold_train, y_fold_train.values.ravel())
    preds = knn_model.predict(X_fold_val)
    xr = preds.shape[0]
    #print(preds.shape, y_fold_val.shape)
    preds = preds.reshape(xr,1)
    #print(preds.shape)
    acc = np.sum(preds==y_fold_val)/len(y_fold_val)
    #acc = calc_accuracy(preds, y[validate])
    temp.append(acc)
  x= sum(temp)/len(temp)
  temp.clear()
  results.append(x)
  i = np.argmax(results)
  nn = neighbor_vals[i]
  #return nn
print(nn)
model = KNeighborsClassifier(n_neighbors=nn)
model.fit(x_training, y_training)
predic = model.predict(x_testing)
predic = predic.reshape(predic.shape[0],1)
accur = np.sum(predic==y_testing)/len(y_testing)
print(accur)


50
Potability    0.652482
dtype: float64


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
